# SVM Genre Classification

In [88]:
# Import libraries and functions

from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

import sklearn
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [55]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [56]:
directory = "/mnt/c/Users/nikhi/Desktop/final_project/genres"
f= open("my1.dat" ,'wb')
f2= open("my2.dat", 'wb')
i=0
for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+"/"+folder):  
        (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)
        feature1 = (mean_matrix, covariance)
        pickle.dump(feature1, f2)
f.close()

In [57]:
# dataset = []
# def loadDataset(filename , split , train , test):
#     with open("my.dat" , 'rb') as f:
#         while True:
#             try:
#                 dataset.append(pickle.load(f))
#             except EOFError:
#                 f.close()
#                 break  
#     for x in range(len(dataset)):
#         if random.random() <split :      
#             train.append(dataset[x])
#         else:
#             test.append(dataset[x])  
# trainingSet = []
# testSet = []
# loadDataset("my.dat" , 0.8, trainingSet, testSet)


def loadDataset(filename, dataset):
    with open(filename , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break   

dataset1 = []      # Dataset with labels 
dataset2 = []      # Dataset without labels
loadDataset("my1.dat", dataset1)
loadDataset("my2.dat", dataset2) 

## Exploratory data analysis

In [72]:
dataset1_np = np.array(dataset1) 
print(dataset1_np.shape)

print(len(dataset2))

# Current shape of dataset2: 1000 elements each containing 2 elements: #1: 13 elements #2: 13 elements each containing 13 elements
print(len(dataset2[0][1][1]))

# Declare a new numpy array for dataset2
dataset2_np = np.empty((1000, 182))

for i in range(len(dataset2)):
    # Declare a temp array that we will append to the main np array
    array_i = []
    for j in range(len(dataset2[i][0])):
        array_i.append(dataset2[i][0][j])
    for k in range(len(dataset2[i][1])):
        for k2 in range(len(dataset2[i][1][1])):
            array_i.append(dataset2[i][1][k][k2])
    array_np = np.array(array_i)
    np.append(dataset2_np, array_np)


print(dataset2_np.shape)

# Preparing Target Data
target = []
for i in range(0, 1000):
    target.append(dataset1_np[i][2])
target_np = np.array(target)

print(np.unique(target_np))


(1000, 3)
1000
13
(1000, 182)
[ 1  2  3  4  5  6  7  8  9 10]


/home/nikhilbhave9/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


There are 10 genres and each genre has a 100 songs each. Thus, there are a 1000 audio samples in total. 
Each song has 3 "features" associated with it associated with it. Each of these 3 features is represented using a tuple of varying sizes. 
- Feature #1: Mean_matrix -> A flat array of size 13
- Feature #2: Covariance  -> A 2-D matrix of size 13 x 13
- Feature #3: Label       -> An int representing the correct label for the audio sample 

However, we do not need the labels "feature" while fitting an SVM model to our data. We extract the labels from this dataset and add them to a "target" array. 

Now, we create another dataset, called dataset2, without labels as the third feature. In order to work with this second dataset and feed it to .fit method, we first need to flatten the data. Thus, we will now have, for each of the 1000 songs, one flat 1-D array of size 13 + (13 * 13) = 13 + 169 = 182. Thus, our final dataset2 shape should be (1000, 182)


### Cleaning the dataset 
Let's check for inconsistent values

In [84]:
dataset2_np.shape[0]
dataset2_np_copy = dataset2_np

In [85]:
for n in range(0, dataset2_np.shape[0]):
    for m in range(len(dataset2_np[n])):
        if np.isnan(dataset2_np[n][m]):
            dataset2_np[n][m] = 0.0


In [86]:
# Splice the dataset into X => 2 main features and Y => Correct labels
X_train, X_test, y_train, y_test = train_test_split(dataset2_np, target_np, test_size=0.2,random_state=109) # 80% training and 20% test

# Print shape of X_train 
print("Shape of X_train: ")
print(X_train.shape)

# Print shape of X_test 
print("Shape of X_test: ")
print(X_test.shape)

# Print shape of y_train 
print("Shape of y_train: ")
print(y_train.shape)

# Print shape of y_test 
print("Shape of y_test: ")
print(y_test.shape)

Shape of X_train: 
(800, 182)
Shape of X_test: 
(200, 182)
Shape of y_train: 
(800,)
Shape of y_test: 
(200,)


## Fitting our data to an SVM model

In [87]:
#Create SVM Classifier
classifier = svm.SVC(kernel='rbf')

#Train the model using the training sets
classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = classifier.predict(X_test)

/home/nikhilbhave9/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/nikhilbhave9/.local/lib/python3.6/site-packages/numpy/core/_methods.py:192: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)
/home/nikhilbhave9/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## Model Accuracy

In [92]:
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", metrics.recall_score(y_test, y_pred,average="micro"))

Accuracy: 0.09
Precision: 0.09
Recall: 0.09
